In [1]:
import torch
from transformer import SyntheticSequenceDataset, TinyTransformer, generate_square_subsequent_mask
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import time

from train_model import train_model, test_model, rl_model

Using cuda


In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using {device}')

Using cuda


In [ ]:
batch_size = 1024
rl_batch_size = 1024

vocab_size = 6
seq_len = 8
epochs = 1000
batch_size = 1024
d_model = 3
nhead = 3
num_layers = 5
dim_ff = 2
lr = 1e-2
model = train_model(vocab_size, seq_len, epochs, batch_size, d_model, nhead, num_layers, dim_ff, lr, device, verbose=True)
base_model = TinyTransformer(vocab_size, d_model, nhead, num_layers, dim_ff, seq_len)   # same arch as base_model
base_model.load_state_dict(model.state_dict())  # copy weights
for p in base_model.parameters():
    p.requires_grad_(False)


c:\Users\User\source\repos\Arkleseisure\SimulatorToyModel\.toysimulatorvenve\Lib\site-packages\torch\nn\modules\transformer.py:392: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.num_heads is odd
  warnings.warn(


Model param count: 451
Model count by layer:
('pos_emb', Parameter containing:
tensor([[[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]], device='cuda:0', requires_grad=True))
('embedding.weight', Parameter containing:
tensor([[ 0.5077,  0.5392, -0.9084],
        [-0.7566,  0.7558, -0.6125],
        [-0.1811, -1.5659,  0.5352],
        [-0.3716, -1.5232, -0.8678],
        [-0.5672,  1.5776, -1.7446],
        [ 0.9225,  0.0418,  0.3847]], device='cuda:0', requires_grad=True))
('encoder.layers.0.self_attn.in_proj_weight', Parameter containing:
tensor([[-0.6608,  0.1597, -0.5351],
        [ 0.6106,  0.2427,  0.3646],
        [ 0.1721,  0.3995,  0.1437],
        [ 0.1274, -0.5166,  0.2849],
        [-0.6803, -0.6468,  0.6665],
        [ 0.1093, -0.0715,  0.6282],
        [-0.5804, -0.3331, -0.1550],
        [ 0.4310,  0.3538, -0.3077],
        [-0.2547, -0.2200, -0

loss: 0.37768346071243286:  85%|████████▌ | 851/1000 [00:25<00:04, 35.52it/s]

In [ ]:
base_model = base_model.to(device)
test_model(model, batch_size, seq_len, vocab_size)
test_model(base_model, batch_size, seq_len, vocab_size)

TinyTransformer(
  (embedding): Embedding(6, 3)
  (encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-4): 5 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=3, out_features=3, bias=True)
        )
        (linear1): Linear(in_features=3, out_features=2, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=2, out_features=3, bias=True)
        (norm1): LayerNorm((3,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((3,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (head): Linear(in_features=3, out_features=6, bias=True)
)
input : [5, 2, 1, 1, 0, 1, 1]
preds : [3, 1, 1, 0, 1, 1, 0]
reward: 0.5714285714285714
logits 0: [-13.0758638381958, 2.4244611263275146, 7.17145299911499, 7.2258830070495605, 7.179135322570801, -1.03305

In [ ]:
model = rl_model(base_model, rl_batch_size=rl_batch_size, verbose=True)

loss=-0.0459  avg_reward=0.874 KL=0.303: 100%|██████████| 1000/1000 [01:01<00:00, 16.27it/s]


In [ ]:
test_model(model)

TinyTransformer(
  (embedding): Embedding(6, 3)
  (encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-4): 5 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=3, out_features=3, bias=True)
        )
        (linear1): Linear(in_features=3, out_features=2, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=2, out_features=3, bias=True)
        (norm1): LayerNorm((3,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((3,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (head): Linear(in_features=3, out_features=6, bias=True)
)
input : [5, 2, 1, 1, 0, 1, 1]
preds : [4, 1, 1, 0, 1, 1, 0]
reward: 0.5714285714285714
logits 0: [-6.792204856872559, -0.9793281555175781, 5.091799259185791, 5.110385894775391, 5.141311168670654, -0.840

In [ ]:
num_layers = 2
d_model = 3
nhead = 3
dim_ff = 2
epochs = 500
base_model = train_model(vocab_size, seq_len, epochs, batch_size, d_model, nhead, num_layers, dim_ff, lr, device, verbose=True)
model = rl_model(base_model, rl_batch_size=rl_batch_size, verbose=True, beta=2)
test_model(base_model, batch_size, seq_len, vocab_size)
test_model(model, batch_size, seq_len, vocab_size)

Model param count: 220
Model count by layer:
('pos_emb', Parameter containing:
tensor([[[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]], device='cuda:0', requires_grad=True))
('embedding.weight', Parameter containing:
tensor([[ 0.6441,  0.9567, -0.7844],
        [ 0.4242,  0.6012,  0.6982],
        [-0.3143, -0.2192, -0.1752],
        [-0.1912, -2.8662,  1.0095],
        [ 0.5639,  0.9740,  0.8723],
        [-0.2110, -0.3603, -1.0006]], device='cuda:0', requires_grad=True))
('encoder.layers.0.self_attn.in_proj_weight', Parameter containing:
tensor([[ 0.2194,  0.1722, -0.6215],
        [ 0.3079, -0.6127,  0.2562],
        [-0.1280, -0.0357, -0.5009],
        [ 0.4075,  0.0641, -0.4380],
        [ 0.6149, -0.3556,  0.4145],
        [ 0.3789, -0.5826, -0.4429],
        [ 0.1451, -0.5592, -0.2609],
        [ 0.6689, -0.0197, -0.3486],
        [-0.0600, -0.5176, -0

loss=-0.0031  avg_reward=0.533 KL=0.018: 100%|██████████| 1000/1000 [00:27<00:00, 36.07it/s]


TinyTransformer(
  (embedding): Embedding(6, 3)
  (encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-1): 2 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=3, out_features=3, bias=True)
        )
        (linear1): Linear(in_features=3, out_features=2, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=2, out_features=3, bias=True)
        (norm1): LayerNorm((3,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((3,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (head): Linear(in_features=3, out_features=6, bias=True)
)
input : [5, 4, 1, 1, 0, 1, 0]
preds : [4, 1, 1, 1, 1, 0, 1]
logits 0: [-4.60508394241333, -2.3911492824554443, 2.9826507568359375, 2.993257999420166, 3.008551597595215, -2.30002498626709]
probs 0: [0.00

In [ ]:
def max_ones(output_tokens: torch.LongTensor):
    # output_tokens: (B, T)
    rewards = (output_tokens == 1).mean(dim=1, dtype=float)
    return rewards

def max_zeros(output_tokens: torch.LongTensor):
    # output_tokens: (B, T)
    rewards = (output_tokens == 0).mean(dim=1, dtype=float)
    return rewards

def max_same(output_tokens: torch.LongTensor):
    # output_tokens: (B, T)
    rewards = torch.zeros(output_tokens.size(0), dtype=float, device=output_tokens.device)
    for i in range(output_tokens.size(0)):
        counts = torch.bincount(output_tokens[i])
        rewards[i] = counts.max().item() / output_tokens.size(1)
    return rewards

def max_alternate(output_tokens: torch.LongTensor):
    # output_tokens: (B, T)
    # Vectorized: count positions where adjacent tokens differ
    diffs = (output_tokens[:, 1:] != output_tokens[:, :-1]).float()
    rewards = diffs.mean(dim=1)
    return rewards

def max_palindrome(output_tokens: torch.LongTensor):
    # output_tokens: (B, T)
    # Vectorized: compare first half with reversed second half
    T = output_tokens.size(1)
    first_half = output_tokens[:, :T//2]
    second_half = output_tokens[:, -T//2:].flip(dims=[1])
    matches = (first_half == second_half).float()
    rewards = matches.mean(dim=1)
    return rewards

def max_ascending(output_tokens: torch.LongTensor):
    # output_tokens: (B, T)
    # Reward sequences where each token >= previous token
    ascending = (output_tokens[:, 1:] >= output_tokens[:, :-1]).float()
    rewards = ascending.mean(dim=1)
    return rewards

def max_descending(output_tokens: torch.LongTensor):
    # output_tokens: (B, T)
    # Reward sequences where each token <= previous token
    descending = (output_tokens[:, 1:] <= output_tokens[:, :-1]).float()
    rewards = descending.mean(dim=1)
    return rewards

def max_unique(output_tokens: torch.LongTensor):
    # output_tokens: (B, T)
    # Reward diversity - fraction of unique tokens
    rewards = torch.zeros(output_tokens.size(0), dtype=float, device=output_tokens.device)
    for i in range(output_tokens.size(0)):
        rewards[i] = torch.unique(output_tokens[i]).size(0) / output_tokens.size(1)
    return rewards

def max_even(output_tokens: torch.LongTensor):
    # output_tokens: (B, T)
    # Reward even-valued tokens
    rewards = (output_tokens % 2 == 0).float().mean(dim=1)
    return rewards

def max_odd(output_tokens: torch.LongTensor):
    # output_tokens: (B, T)
    # Reward odd-valued tokens
    rewards = (output_tokens % 2 == 1).float().mean(dim=1)
    return rewards

def max_runs(output_tokens: torch.LongTensor):
    # output_tokens: (B, T)
    # Reward long runs of same token (minimize transitions)
    transitions = (output_tokens[:, 1:] != output_tokens[:, :-1]).float()
    rewards = 1.0 - transitions.mean(dim=1)
    return rewards

def max_boundary_match(output_tokens: torch.LongTensor):
    # output_tokens: (B, T)
    # Reward when first and last tokens match
    rewards = (output_tokens[:, 0] == output_tokens[:, -1]).float()
    return rewards

In [ ]:
reward_fns = [max_ones, max_zeros, max_same, max_alternate, max_palindrome, max_ascending, max_descending, max_unique, max_even, max_odd, max_runs, max_boundary_match]
reward_fn_names = ['ones', 'zeros', 'same', 'alternate', 'palindrome', 'ascending', 'descending', 'unique', 'even', 'odd', 'runs', 'boundary_match']

test = torch.tensor([[5]], device=device)
model_probs = [torch.tensor([[0 for i in range(6)]], dtype=float, device=device) for _ in range(len(reward_fns))]
base_model_probs = [torch.tensor([[0 for i in range(6)]], dtype=float, device=device) for _ in range(len(reward_fns))]

def quick_test_model(model, model_probs):
    model_logits = model(test)
    p_m = torch.softmax(model_logits, dim=-1)
    return torch.cat((model_probs, p_m[0, :, :]))

beta=1

for epoch in tqdm(range(1000)):
    for i, reward_fn in enumerate(reward_fns):
        base_model = train_model(batch_size=batch_size)
        model = rl_model(base_model, rl_batch_size=rl_batch_size, beta=beta, reward_fn=reward_fn)

        model_probs[i] = quick_test_model(model, model_probs[i])
        base_model_probs[i] = quick_test_model(base_model, base_model_probs[i])

        model_means = model_probs[i][1:].mean(dim=0)
        base_model_means = base_model_probs[i][1:].mean(dim=0)
        model_stds = model_probs[i][1:].std(dim=0)
        base_model_stds = base_model_probs[i][1:].std(dim=0)

        text = f'Epoch: {epoch+1}\nModel mean:{model_means}\nModel std:{model_stds}\nBase Model means:{base_model_means}\nBase Model stds:{base_model_stds}'
        with open(f'{reward_fn_names[i]}_beta_{beta}_log_file.txt', 'w') as f:
            f.write(text)

  0%|          | 0/1000 [00:00<?, ?it/s]C:\Users\User\AppData\Local\Temp\ipykernel_27128\2220080227.py:25: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\aten\src\ATen\native\ReduceOps.cpp:1857.)
  model_stds = model_probs[i][1:].std(dim=0)
C:\Users\User\AppData\Local\Temp\ipykernel_27128\2220080227.py:26: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\aten\src\ATen\native\ReduceOps.cpp:1857.)
  base_model_stds = base_model_probs[i][1:].std(dim=0)
 42%|████▏     | 423/1000 [158:02:32<131:55:41, 823.12s/it]   

In [48]:
for p in model.named_parameters():
    print(p[0])
    print(p[1])

pos_emb
Parameter containing:
tensor([[[ 0.4930, -0.2230, -0.1690],
         [ 0.8037, -0.6332,  0.2778],
         [ 0.8064, -0.2626,  0.0808],
         [-1.0894, -0.2210,  0.1107],
         [-0.5729, -0.3024,  0.0262],
         [ 0.7611, -0.0107,  0.2603],
         [-0.9652, -0.0672,  0.0034],
         [ 0.0000,  0.0000,  0.0000]]], device='cuda:0', requires_grad=True)
embedding.weight
Parameter containing:
tensor([[-1.2440,  0.1865,  1.0439],
        [ 0.1008,  0.5810,  1.3011],
        [ 0.7975,  0.8791,  1.2768],
        [ 2.5740,  1.5790, -1.6963],
        [-1.3601, -0.9434,  3.0559],
        [-0.6374, -0.5944, -1.0413]], device='cuda:0', requires_grad=True)
encoder.layers.0.self_attn.in_proj_weight
Parameter containing:
tensor([[-0.3988,  0.7636,  1.7770],
        [-1.7054, -0.5856, -0.5206],
        [-1.8437, -0.1521, -0.7063],
        [ 0.0672, -1.9782,  0.0395],
        [-1.1515, -0.1998,  1.6773],
        [-1.3865,  1.8502,  0.5987],
        [-0.1469, -0.9062,  0.9657],
     

In [6]:
from train_model import test_model
beta = 1
reward_fn = max_boundary_match
base_model = train_model(batch_size=batch_size, verbose = True)
model = rl_model(base_model, rl_batch_size=rl_batch_size, beta=beta, reward_fn=reward_fn, verbose=True)

test_model(base_model, reward_fn=reward_fn)
test_model(model, reward_fn=reward_fn)

Model param count: 220
Model count by layer:
('pos_emb', Parameter containing:
tensor([[[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]], device='cuda:0', requires_grad=True))
('embedding.weight', Parameter containing:
tensor([[ 0.5798,  0.3730, -1.1778],
        [ 1.0841, -0.7502, -0.1226],
        [ 0.6569, -1.4457, -0.1741],
        [-1.1094, -0.1093,  1.1265],
        [-0.0614,  0.2181, -0.6663],
        [ 0.4830,  2.2888,  0.1088]], device='cuda:0', requires_grad=True))
('encoder.layers.0.self_attn.in_proj_weight', Parameter containing:
tensor([[ 0.6913,  0.3412,  0.1481],
        [ 0.2179,  0.6534, -0.4690],
        [-0.4616, -0.5935, -0.1431],
        [ 0.0079, -0.4998, -0.4548],
        [ 0.6591,  0.0831,  0.1527],
        [ 0.0903,  0.1182,  0.1404],
        [ 0.4189, -0.2423,  0.4320],
        [ 0.5616, -0.0173, -0.1032],
        [ 0.2046,  0.5215,  0

loss=-0.0164  avg_reward=0.528 KL=0.000:   0%|          | 3/1000 [00:00<01:13, 13.65it/s]

[0, 2, 1, 1, 0, 1, 1, 0] Reward: 1.0
[0, 2, 1, 1, 0, 1, 1, 0] Reward: 1.0
[0, 4, 1, 1, 1, 1, 0, 0] Reward: 1.0
[0, 3, 0, 0, 1, 0, 0, 1] Reward: 0.0


loss=-0.0173  avg_reward=0.523 KL=0.000:   1%|          | 8/1000 [00:00<00:53, 18.45it/s]

[0, 2, 1, 1, 0, 1, 1, 0] Reward: 1.0
[0, 2, 1, 1, 0, 1, 1, 0] Reward: 1.0
[0, 2, 1, 1, 0, 1, 1, 0] Reward: 1.0
[0, 2, 1, 1, 0, 1, 1, 0] Reward: 1.0
[0, 3, 0, 0, 1, 0, 0, 1] Reward: 0.0


...

loss=-0.0553  avg_reward=0.861 KL=0.038: 100%|█████████▉| 995/1000 [00:43<00:00, 23.25it/s]

[0, 3, 0, 0, 1, 0, 0, 1] Reward: 0.0
[0, 2, 1, 1, 0, 1, 1, 0] Reward: 1.0
[0, 2, 1, 1, 0, 1, 1, 0] Reward: 1.0
[0, 3, 0, 0, 1, 0, 0, 1] Reward: 0.0
[0, 2, 1, 1, 0, 1, 1, 0] Reward: 1.0


loss=-0.0693  avg_reward=0.835 KL=0.038: 100%|██████████| 1000/1000 [00:43<00:00, 23.06it/s]

[0, 2, 1, 1, 0, 1, 1, 0] Reward: 1.0
[0, 3, 0, 0, 1, 0, 0, 1] Reward: 0.0
[0, 4, 1, 0, 1, 0, 0, 0] Reward: 1.0
[0, 2, 1, 1, 0, 1, 1, 0] Reward: 1.0
[0, 3, 0, 0, 1, 0, 0, 1] Reward: 0.0
TinyTransformer(
  (embedding): Embedding(6, 3)
  (encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-1): 2 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=3, out_features=3, bias=True)
        )
        (linear1): Linear(in_features=3, out_features=2, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=2, out_features=3, bias=True)
        (norm1): LayerNorm((3,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((3,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (head): Linear(in_features=3, out_features=6, bias=True)
)
input : [5, 3, 0,

input : [5, 3, 0, 0, 1, 0, 0]
preds : [2, 0, 0, 1, 0, 0, 1]
reward: 0.0
logits 0: [-0.5867127776145935, -6.600326061248779, 4.210480213165283, 4.172505855560303, 4.068892955780029, -0.8872502446174622]
probs 0: [0.002900733845308423, 7.092981832101941e-06, 0.3514813780784607, 0.33838436007499695, 0.3050786256790161, 0.002147761406376958]
logits 1: [7.593873977661133, 0.07993529736995697, -1.3573397397994995, -1.3466355800628662, -2.4212021827697754, -5.511781215667725]
probs 1: [0.9991480112075806, 0.0005449637537822127, 0.0001294693793170154, 0.00013086266699247062, 4.468252882361412e-05, 2.0319635041232686e-06]
logits 2: [7.716328144073486, 1.9825786352157593, -2.5442287921905518, -2.3988771438598633, -3.598855495452881, -5.15521240234375]
probs 2: [0.9966860413551331, 0.0032242052257061005, 3.48702487826813e-05, 4.032555079902522e-05, 1.2146091648901347e-05, 2.561648670962313e-06]
logits 3: [1.000657081604004, 9.1657075881958, -5.728024959564209, -4.681591033935547, -5.8636198043823

In [16]:
# Cell 1 - install (run once if you don't have plotly)
# !pip install plotly

import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from typing import Optional, Sequence, Union
import plotly.io as pio

pio.renderers.default = "notebook" # "iframe_connected"   or "notebook", "notebook_connected", "browser"

def plot_3d_points(
    points: Union[np.ndarray, Sequence[Sequence[float]]],
    labels: Optional[Union[Sequence[str], Sequence[int]]] = None,
    title: Optional[str] = None,
    marker_size: int = 5,
    color_map: Optional[Sequence[str]] = None,
    show_labels_as_text: bool = False,
    hover_template: str = "<b>%{text}</b><br>x=%{x:.3f}<br>y=%{y:.3f}<br>z=%{z:.3f}<extra></extra>",
    renderer: Optional[str] = None,
    save_html: Optional[str] = None
):
    """
    points: N x 3 array-like (x,y,z)
    labels: optional list of labels (strings or ints) length N. If provided, points are colored by label.
    show_labels_as_text: if True, shows the label as on-plot text (can overlap).
    renderer: optional plotly renderer (e.g. 'notebook', 'iframe', 'iframe_connected', 'notebook_connected')
    save_html: if given, writes an interactive HTML file to that path
    """
    pts = np.asarray(points)
    if pts.ndim != 2 or pts.shape[1] != 3:
        raise ValueError("points must be shape (N,3)")

    n = pts.shape[0]
    x, y, z = pts[:,0], pts[:,1], pts[:,2]

    if labels is None:
        # single color
        trace = go.Scatter3d(
            x=x, y=y, z=z,
            mode = "markers+text" if show_labels_as_text else "markers",
            marker = dict(size=marker_size, opacity=0.9),
            text = None,
            hovertext = [f"pt {i}" for i in range(n)],
            hovertemplate = hover_template
        )
        fig = go.Figure(trace)
    else:
        labels.append('origin')
        x = np.append(x, 0)
        y = np.append(y, 0)
        z = np.append(z, 0)
        labels_arr = np.asarray(labels)
        # map labels to discrete colors
        unique = np.unique(labels_arr)
        # get a palette large enough
        palette = list(px.colors.qualitative.Plotly)
        if color_map is not None:
            palette = list(color_map)
        # repeat palette if necessary
        while len(palette) < len(unique):
            palette = palette + palette
        color_for = {lab: palette[i] for i, lab in enumerate(unique)}
        # build traces (one trace per label) to get clean legend & hover
        traces = []
        for lab in unique:
            mask = labels_arr == lab
            txt = labels_arr[mask].astype(str) if show_labels_as_text else None
            hover = [f"{lab} — idx {i}" for i in np.where(mask)[0]]
            traces.append(
                go.Scatter3d(
                    x = x[mask], y = y[mask], z = z[mask],
                    mode = "markers+text" if show_labels_as_text else "markers",
                    marker = dict(size=marker_size),
                    name = str(lab),
                    text = txt,
                    hovertext = hover,
                    hovertemplate = hover_template
                )
            )
        fig = go.Figure(data=traces)

    # layout: decent camera and axes labels
    fig.update_layout(
        title = title or "",
        scene = dict(
            xaxis_title="x",
            yaxis_title="y",
            zaxis_title="z",
            aspectmode="auto",
        ),
        margin = dict(l=0, r=0, t=30, b=0),
        legend = dict(itemsizing="constant")
    )

    # set a nice initial camera angle (optional)
    fig.update_layout(scene_camera=dict(
        eye=dict(x=1.25, y=1.25, z=1.0)
    ))

    # show with requested renderer if provided
    if renderer:
        fig.show(renderer=renderer)
    else:
        fig.show()

    if save_html:
        fig.write_html(save_html, include_plotlyjs='cdn')



In [8]:
pts = {}
for p in model.named_parameters():
    print(p[0])
    print(p[1].detach().cpu().numpy().shape)
    pts[p[0]] = p[1].detach().cpu().numpy()

pos_emb
(1, 8, 3)
embedding.weight
(6, 3)
encoder.layers.0.self_attn.in_proj_weight
(9, 3)
encoder.layers.0.self_attn.in_proj_bias
(9,)
encoder.layers.0.self_attn.out_proj.weight
(3, 3)
encoder.layers.0.self_attn.out_proj.bias
(3,)
encoder.layers.0.linear1.weight
(2, 3)
encoder.layers.0.linear1.bias
(2,)
encoder.layers.0.linear2.weight
(3, 2)
encoder.layers.0.linear2.bias
(3,)
encoder.layers.0.norm1.weight
(3,)
encoder.layers.0.norm1.bias
(3,)
encoder.layers.0.norm2.weight
(3,)
encoder.layers.0.norm2.bias
(3,)
encoder.layers.1.self_attn.in_proj_weight
(9, 3)
encoder.layers.1.self_attn.in_proj_bias
(9,)
encoder.layers.1.self_attn.out_proj.weight
(3, 3)
encoder.layers.1.self_attn.out_proj.bias
(3,)
encoder.layers.1.linear1.weight
(2, 3)
encoder.layers.1.linear1.bias
(2,)
encoder.layers.1.linear2.weight
(3, 2)
encoder.layers.1.linear2.bias
(3,)
encoder.layers.1.norm1.weight
(3,)
encoder.layers.1.norm1.bias
(3,)
encoder.layers.1.norm2.weight
(3,)
encoder.layers.1.norm2.bias
(3,)
head.weigh

In [9]:
import torch
import numpy as np

def tensor_to_points(name, t, num_heads=3, d_model=3):
    """
    Convert a tensor into points of shape (n, 3).
    Handles:
      - pos_emb (1, seq, 3)
      - embedding weights (vocab, 3)
      - linear weights (out, in)
      - biases (n,)
      - MultiheadAttention QKV weights (9, 3) → per-head Q/K/V points
      - MultiheadAttention QKV bias (9,) → per-head Q/K/V points
    """
    t = t.detach().cpu()

    # --- special case: MultiheadAttention QKV weights ---
    if "in_proj_weight" in name:
        # shape = (3 * d_model, d_model) = (9, 3)
        assert t.shape == (3 * d_model, d_model)

        # split Q, K, V along rows
        Wq, Wk, Wv = t.chunk(3, dim=0)    # each (3, 3)

        head_dim = d_model // num_heads   # here = 1

        # reshape to (num_heads, head_dim, d_model) = (3, 1, 3)
        Wq_h = Wq.view(num_heads, head_dim, d_model)
        Wk_h = Wk.view(num_heads, head_dim, d_model)
        Wv_h = Wv.view(num_heads, head_dim, d_model)

        out = {}
        for h in range(num_heads):
            out[f"{name}.Q.head{h}"] = Wq_h[h].numpy()   # (1,3)
            out[f"{name}.K.head{h}"] = Wk_h[h].numpy()   # (1,3)
            out[f"{name}.V.head{h}"] = Wv_h[h].numpy()   # (1,3)
        return out

    # --- special case: MultiheadAttention QKV bias ---
    if "in_proj_bias" in name:
        # shape = (9,)
        assert t.shape == (3 * d_model,)
        bq, bk, bv = t.chunk(3, dim=0)    # each (3,)
        out = {}
        for part_name, vec in zip(["Q", "K", "V"], [bq, bk, bv]):
            # reshape each bias vector (3,) as (3,1) then broadcast to (3,3)
            # so each element becomes a separate 3D point
            pts = vec.view(-1, 1).expand(-1, 3)
            out[f"{name}.{part_name}"] = pts.numpy()     # (3,3)
        return out

    # --- normal 3D-shaped positional embeddings ---
    if t.ndim == 3:
        # expect (1, seq, 3)
        assert t.shape[-1] == 3
        return {name: t.reshape(-1, 3).numpy()}

    # --- 2D matrices: weights ---
    if t.ndim == 2:
        out_dim, in_dim = t.shape
        if in_dim == 3:
            # each row is a 3D vector → (out_dim, 3)
            return {name: t.numpy()}
        elif out_dim == 3:
            # transpose so rows become 3D
            return {name: t.T.numpy()}
        else:
            # flatten weird shapes into 3D by grouping triples
            flat = t.flatten()
            n = flat.numel() // 3
            return {name: flat[:3*n].view(n, 3).numpy()}

    # --- 1D tensors: biases or norm weights ---
    if t.ndim == 1:
        if t.numel() == 3:
            # (3,) → (1,3)
            return {name: t.view(1,3).numpy()}
        else:
            # turn N numbers into N 3D points by repeating each scalar 3 times
            pts = t.view(-1, 1).expand(-1, 3)   # (N,3)
            return {name: pts.numpy()}

    raise ValueError(f"Don't know how to convert tensor {name} with shape {t.shape}")


def extract_all_points(model, num_heads=3, d_model=3):
    """
    Walks every named parameter and buffer in the model,
    converting them to 3D point clouds.
    """
    results = {}

    # named parameters
    for name, p in model.named_parameters():
        out = tensor_to_points(name, p, num_heads=num_heads, d_model=d_model)
        results.update(out)

    # named buffers (e.g. pos_emb)
    for name, b in model.named_buffers():
        out = tensor_to_points(name, b, num_heads=num_heads, d_model=d_model)
        results.update(out)

    return results


In [38]:
all_pts_model = extract_all_points(model, num_heads=model.nhead, d_model=model.d_model)
all_pts_base_model = extract_all_points(base_model, num_heads=base_model.nhead, d_model=base_model.d_model) 
for key in all_pts_model.keys():
    print(f"{key}: model pts={all_pts_model[key].shape}, base_model pts={all_pts_base_model[key].shape}")

for layer in model.named_parameters():
    print(layer[0], layer[1].detach().cpu().numpy().shape)

pos_emb: model pts=(8, 3), base_model pts=(8, 3)
embedding.weight: model pts=(6, 3), base_model pts=(6, 3)
encoder.layers.0.self_attn.in_proj_weight.Q.head0: model pts=(1, 3), base_model pts=(1, 3)
encoder.layers.0.self_attn.in_proj_weight.K.head0: model pts=(1, 3), base_model pts=(1, 3)
encoder.layers.0.self_attn.in_proj_weight.V.head0: model pts=(1, 3), base_model pts=(1, 3)
encoder.layers.0.self_attn.in_proj_weight.Q.head1: model pts=(1, 3), base_model pts=(1, 3)
encoder.layers.0.self_attn.in_proj_weight.K.head1: model pts=(1, 3), base_model pts=(1, 3)
encoder.layers.0.self_attn.in_proj_weight.V.head1: model pts=(1, 3), base_model pts=(1, 3)
encoder.layers.0.self_attn.in_proj_weight.Q.head2: model pts=(1, 3), base_model pts=(1, 3)
encoder.layers.0.self_attn.in_proj_weight.K.head2: model pts=(1, 3), base_model pts=(1, 3)
encoder.layers.0.self_attn.in_proj_weight.V.head2: model pts=(1, 3), base_model pts=(1, 3)
encoder.layers.0.self_attn.in_proj_bias.Q: model pts=(3, 3), base_model pt

In [37]:
'''
for pt in all_pts_base_model.keys():
    print(pt, all_pts_base_model[pt].shape)
    print((all_pts_base_model[pt] - all_pts_model[pt]))
'''
    
pt = 'head.'  # change this to visualize other parts of the model

if 'encoder' or 'head' in pt:
    pts_base = np.array([[]])
    pts = np.array([[]])
    for key in all_pts_model.keys():
        if pt in key:
            print(pts_base)
            if pts_base.size == 0:
                pts_base = all_pts_base_model[key]
                pts = all_pts_model[key]
                labels_base = [key[len(pt):] + '0']
                labels = [key[len(pt):] + '0']
                for i in range(all_pts_base_model[key].shape[0] - 1):
                    labels_base.append(key[len(pt):] + str(i + 1))
                    labels.append(key[len(pt):] + str(i + 1))
            else:
                pts_base = np.append(pts_base, all_pts_base_model[key], axis=0)
                pts = np.append(pts, all_pts_model[key], axis=0)
                print(pts_base.shape)
                for i in range(all_pts_base_model[key].shape[0]):
                    labels_base.append(key[len(pt):] + str(i))
                    labels.append(key[len(pt):] + str(i))
            print(pts_base)
else:
    pts_base = all_pts_base_model[pt]
    pts = all_pts_model[pt]
    labels_base = [str(i) for i in range(len(pts_base))]
    labels = [str(i) for i in range(len(pts))]
print(pts_base.shape)
print(labels)
plot_3d_points(pts_base, labels=labels_base, show_labels_as_text=True, marker_size=8, renderer='notebook_connected')   
plot_3d_points(pts, labels=labels, show_labels_as_text=True, marker_size=8, renderer='notebook_connected')

[]
[[ 1.2978913   0.77315223 -0.46048418]
 [-1.0961275   1.5237862  -0.35797852]
 [-0.31867623 -1.1218554   0.9555952 ]
 [-0.20730558 -0.9290279   1.0864348 ]
 [-0.71706647 -1.4376966   0.59803325]
 [-0.56169474  0.3174633   0.7844257 ]]
[[ 1.2978913   0.77315223 -0.46048418]
 [-1.0961275   1.5237862  -0.35797852]
 [-0.31867623 -1.1218554   0.9555952 ]
 [-0.20730558 -0.9290279   1.0864348 ]
 [-0.71706647 -1.4376966   0.59803325]
 [-0.56169474  0.3174633   0.7844257 ]]
(12, 3)
[[ 1.2978913   0.77315223 -0.46048418]
 [-1.0961275   1.5237862  -0.35797852]
 [-0.31867623 -1.1218554   0.9555952 ]
 [-0.20730558 -0.9290279   1.0864348 ]
 [-0.71706647 -1.4376966   0.59803325]
 [-0.56169474  0.3174633   0.7844257 ]
 [ 0.3175949   0.3175949   0.3175949 ]
 [ 0.46366155  0.46366155  0.46366155]
 [ 0.00154167  0.00154167  0.00154167]
 [ 0.26301378  0.26301378  0.26301378]
 [ 0.00249871  0.00249871  0.00249871]
 [-0.36047423 -0.36047423 -0.36047423]]
(12, 3)
['weight0', 'weight1', 'weight2', 'weight3